# 天猫订单综合分析

通过对订单的数据进行处理，进而分析订单转化率、销量趋势、地域分布等

## 导入库

In [25]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import calendar

## 一、背景及目的

本数据集共收集了发生在一个月内的28010条数据，包含以下：

### 字段说明

1. 订单编号：订单编号
2. 总金额：订单总金额
3. 买家实际支付金额：总金额 - 退款金额（在已付款的情况下）。金额为0（在未付款的情况下）
4. 收货地址：修改为各个省份
5. 订单创建时间：下单时间
6. 订单付款时间：付款时间
7. 退款金额：付款后申请退款的金额。如无付过款，退款金额默认为0

### 分析目的

1. 订单每个环节的转化率
2. 订单成交的时间（按天）趋势(按实际成交）
3. 订单数在地图上的分布

## 二、读取数据

In [5]:
path=r'F:\Data\tmall6650\tmall_order_report.csv'
df=pd.read_csv(path)

## 三、查看数据

In [6]:
df.head()

,订单编号,总金额,买家实际支付金额,收货地址,订单创建时间,订单付款时间,退款金额
0,1,178.8,0.0,上海,2020-02-21 00:00:00,NaN,0.0
1,2,21.0,21.0,内蒙古自治区,2020-02-20 23:59:54,2020-02-21 00:00:02,0.0
2,3,37.0,0.0,安徽省,2020-02-20 23:59:35,NaN,0.0
3,4,157.0,157.0,湖南省,2020-02-20 23:58:34,2020-02-20 23:58:44,0.0
4,5,64.8,0.0,江苏省,2020-02-20 23:57:04,2020-02-20 23:57:11,64.8


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28010 entries, 0 to 28009
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   订单编号      28010 non-null  int64  
 1   总金额       28010 non-null  float64
 2   买家实际支付金额  28010 non-null  float64
 3   收货地址      28010 non-null  object 
 4   订单创建时间    28010 non-null  object 
 5   订单付款时间    24087 non-null  object 
 6   退款金额      28010 non-null  float64
dtypes: float64(3), int64(1), object(3)
memory usage: 1.5+ MB


## 四、数据预处理

### 修改列名

In [8]:
# 查看数据列名
df.columns

Index(['订单编号', '总金额', '买家实际支付金额', '收货地址 ', '订单创建时间', '订单付款时间 ', '退款金额'], dtype='object')

In [9]:
# 去除列名的空格
df.columns=df.columns.str.strip()

In [10]:
# 再次查看列名
df.columns

Index(['订单编号', '总金额', '买家实际支付金额', '收货地址', '订单创建时间', '订单付款时间', '退款金额'], dtype='object')

### 去除重复值

In [11]:
# 查看重复值
df.duplicated().sum()

0

### 去除缺失值

In [12]:
# 查看缺失值
df.isnull().sum()

订单编号           0
总金额            0
买家实际支付金额       0
收货地址           0
订单创建时间         0
订单付款时间      3923
退款金额           0
dtype: int64

这里的付款时间缺失说明是单位统一付款，不需要处理

In [13]:
# 查看数据
df.describe()

,订单编号,总金额,买家实际支付金额,退款金额
count,28010.000000,28010.000000,28010.000000,28010.000000
mean,14005.500000,106.953253,67.921712,20.433271
std,8085.934856,1136.587094,151.493434,71.501963
min,1.000000,1.000000,0.000000,0.000000
25%,7003.250000,38.000000,0.000000,0.000000
50%,14005.500000,75.000000,45.000000,0.000000
75%,21007.750000,119.000000,101.000000,0.000000
max,28010.000000,188320.000000,16065.000000,3800.000000


## 订单转化率-漏斗图

### 计算得出需要的数据

通过对数据进行处理，将得到以下数据
- 总订单数：28010
- 付款订单数：24087
- 到款订单数：18955
- 全额到款订单数：18441

需要注意的是：
1. 买家实际支付金额：总金额 - 退款金额（在已付款的情况下），未付款时金额为0
2. 退款金额：付款后申请退款的金额。未付款时，退款金额为0

### 总订单数

In [14]:
bill_sum=df['订单编号'].count()
print("总订单数：%d" %bill_sum)

总订单数：28010


### 付款订单数

In [15]:
bill_payed_sum=df['订单付款时间'].notnull().sum()
print("付过款的订单数：%d" %bill_payed_sum)

付过款的订单数：24087


### 到款订单数

In [16]:
bill_get_sum=df[(df['总金额']>df['退款金额'])&(df['订单付款时间'].notnull())].shape[0]
print("到款订单数：%d" %bill_get_sum)

到款订单数：18955


### 全额到款订单数

In [17]:
bill_full_sum=df[(df['订单付款时间'].notnull())&(df['退款金额']==0)].shape[0]
print("全额到款订单数：%d" %bill_full_sum)

全额到款订单数：18441


### 数据总结

In [20]:
df_conv=pd.Series({'总订单数':bill_sum,'付款订单数':bill_payed_sum,'到款订单数':bill_get_sum,'全额到款订单数':bill_full_sum})
print(df_conv)

总订单数       28010
付款订单数      24087
到款订单数      18955
全额到款订单数    18441
dtype: int64


### Tableau作图

![](http://m.qpic.cn/psc?/V509KgjP2rVc1x3xXZE72dVD4k46B5pi/ruAMsa53pVQWN7FLK88i5kaehZfZ6lPqVBCCjm8iIgLdGdWAZRgySVIUv.IkQz2GpDrtf*z09vgv9NEiy1n2HCWSxWSsh0m7B1dsGLBYm9E!/b&bo=wwMkAgAAAAADB8Q!&rf=viewer_4)

## 整体订单数趋势

![](http://m.qpic.cn/psc?/V509KgjP2rVc1x3xXZE72dVD4k46B5pi/ruAMsa53pVQWN7FLK88i5tH9AcfzZ1icQQ64lg4f8ukS9Y1Y13NklzXWa46nnLeju4f5giz3EhtcIAtZu*X0fCirToqkvLrbRNjg0mnfdvk!/b&bo=zQP7AQAAAAADBxY!&rf=viewer_4)

## 销量区域分布-地理图

![](http://m.qpic.cn/psc?/V509KgjP2rVc1x3xXZE72dVD4k46B5pi/45NBuzDIW489QBoVep5mcecUZGX5DyrCUqJtrPnrFoOHFwEpYL5PAntaqUUz3WKU*DlbNkFDyq1DX9ZjFoy0xfc8RjKgJYmZKAIPWHLOcc8!/b&bo=zAP7AQAAAAADFwc!&rf=viewer_4)

![](http://m.qpic.cn/psc?/V509KgjP2rVc1x3xXZE72dVD4k46B5pi/45NBuzDIW489QBoVep5mcecUZGX5DyrCUqJtrPnrFoPtrmPN1FtCdPu0ZTSt2KkcNODUftx28nthU72hf2KmXtwu0NMffe9a3L08qLVoNqc!/b&bo=wwP7AQAAAAADJzg!&rf=viewer_4)